## Section 1: Import Libraries and load cifar10 dataset.

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt
#import additional libraries if needed
from sklearn.model_selection import KFold

In [2]:

def load_cifar10_data(batch_size):
# load both training and test datasets, and transform them to tensors.
  transform = transforms.Compose(
      [
          transforms.Resize(256),
          transforms.CenterCrop(224),
          transforms.ToTensor(),
          transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
      ]
  )

  trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                          download=True, transform=transform)
  trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                            shuffle=True, num_workers=2)

  testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)
  testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                          shuffle=False, num_workers=2)
  
  return trainloader, testloader

In [3]:
batch_size = 4
trainloader, testloader = load_cifar10_data(batch_size)
classes = ('plane', 'car', 'bird', 'cat',
            'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

## Check the dimensions of a batch:
for images, labels in trainloader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

Files already downloaded and verified
Files already downloaded and verified
Image batch dimensions: torch.Size([4, 3, 224, 224])
Image label dimensions: torch.Size([4])


## Section 2: Load Pre-trained Model


In [4]:
from torchvision import models
import torch.nn as nn

output_layer_index = 6
no_of_cifar_classes = 10

def getAlexNetModel():
  alexnet =  models.alexnet(weights=models.AlexNet_Weights.DEFAULT)
  for param in alexnet.parameters():
    param.requires_grad = False
  in_features = alexnet.classifier[output_layer_index].in_features
  alexnet.classifier[output_layer_index] = nn.Linear(in_features, no_of_cifar_classes)
  for param in alexnet.classifier[output_layer_index].parameters():
    param.requires_grad = True
  return alexnet

def getVGGModel():
  VGG =  models.vgg13(weights=models.VGG13_Weights.DEFAULT)
  for param in VGG.parameters():
    param.requires_grad = False
  in_features = VGG.classifier[output_layer_index].in_features
  VGG.classifier[output_layer_index] = nn.Linear(in_features, no_of_cifar_classes)
  for param in VGG.classifier[output_layer_index].parameters():
    param.requires_grad = True
  return VGG

def getResNetModel():
  Resnet_50 = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
  for param in Resnet_50.parameters():
    param.requires_grad = False
  in_features = Resnet_50.fc.in_features
  Resnet_50.fc = nn.Linear(in_features, no_of_cifar_classes)
  for param in Resnet_50.fc.parameters():
    param.requires_grad = True
  return Resnet_50

## Section 3: Define optimizer and loss criterion

In [5]:
from torch import optim
#TODO: Declare your hyperparameters here
random_seed = 153
learning_rate = 0.01
num_epochs = 1
no_of_folds = 10

kfold = KFold(n_splits=no_of_folds, random_state=random_seed, shuffle=True)

np.random.seed(random_seed)

AlexNet_Weights_Path = './alex_net.pt'
VGG_Weights_Path = './vgg_net.pt'
RESNET50_Weights_Path = "./res_net.pt"

# optimizer
optimizer = None
# loss
criterion = nn.CrossEntropyLoss()

In [6]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device:', DEVICE)
# Set my pretrained model to be run on GPU
# model = model.to(DEVICE)

Device: cuda:0


## Section 4: Train the model

In [7]:
# Helper function to compute the accuracy
def compute_accuracy(model, testloader):
    correct, total_loss = 0, 0

    with torch.no_grad():
        for data in testloader:
            inputs, labels = data[0].to(DEVICE), data[1].to(DEVICE)
            pred = model(inputs)
            total_loss += criterion(pred, labels).item()
            correct += (pred.argmax(1) == labels).type(torch.float).sum().item()
    
    total_loss /= len(testloader)
    correct /= len(testloader.dataset)
    print(f"For Current Run: Accuracy: {(100*correct):>0.1f}%, Avg loss: {total_loss:>8f}\n")
    return 100*correct

In [8]:
# Helper function to compute the loss over epochs
def compute_epoch_loss(model, trainloader, loss):
    for batch, data in enumerate(trainloader,0):
        # predictions  
        inputs, labels = data[0].to(DEVICE), data[1].to(DEVICE) 
        pred = model(inputs)
        lf = criterion(pred, labels)
        
        # backprop
        loss.zero_grad()
        lf.backward()
        loss.step()

        if batch%100 == 0:
            lf, current = lf.item(), batch * len(inputs)
            print(f"Current loss: {lf:>7f}, [{current:>5d}/{len(trainloader.dataset):>5d}]")

In [9]:
def train_model(epoch_list, accuracy_list, num_epochs, trainloader, testloader, model, optimizer):
    for i in range(num_epochs):
        compute_epoch_loss(model, trainloader, optimizer)
        accuracy = compute_accuracy(model, testloader)
        epoch_list.append(i)
        accuracy_list.append(accuracy)

In [10]:
def perform_kfold_run(model, save_to_location):
    current_model = model()
    current_model.to(DEVICE)
    for i, (train_indices, test_indices) in enumerate(kfold.split(trainloader.dataset)):
        print(f"Fold: {i+1}")
        train_sampler = torch.utils.data.Subset(trainloader.dataset, train_indices)
        test_sampler = torch.utils.data.Subset(trainloader.dataset, test_indices)
        train_dataloader = torch.utils.data.DataLoader(train_sampler, batch_size=45)
        validation_dataloader = torch.utils.data.DataLoader(test_sampler, batch_size=45)
        
        epoch_list = []
        accuracy_list = []

        optimizer = torch.optim.SGD(current_model.parameters(), lr=learning_rate)
        train_model(epoch_list, accuracy_list, num_epochs, train_dataloader, validation_dataloader, current_model, optimizer)

    torch.save(current_model.state_dict(), save_to_location)
    accuracy = compute_accuracy(current_model,testloader)
    print(accuracy)

In [11]:
print("AlexNet")
perform_kfold_run(getAlexNetModel, AlexNet_Weights_Path)

Fold: 1
Current loss: 2.524251, [    0/45000]
Current loss: 0.954922, [ 4500/45000]
Current loss: 0.966442, [ 9000/45000]
Current loss: 0.832314, [13500/45000]
Current loss: 1.016083, [18000/45000]
Current loss: 0.617627, [22500/45000]
Current loss: 0.784167, [27000/45000]
Current loss: 0.769911, [31500/45000]
Current loss: 0.522869, [36000/45000]
Current loss: 0.883095, [40500/45000]
For Current Run: Accuracy: 74.9%, Avg loss: 0.721036

Fold: 2
Current loss: 0.797901, [    0/45000]
Current loss: 0.815104, [ 4500/45000]
Current loss: 0.521414, [ 9000/45000]
Current loss: 0.741547, [13500/45000]
Current loss: 0.820020, [18000/45000]
Current loss: 0.502872, [22500/45000]
Current loss: 0.588301, [27000/45000]
Current loss: 0.751312, [31500/45000]
Current loss: 0.657158, [36000/45000]
Current loss: 0.546398, [40500/45000]
For Current Run: Accuracy: 76.0%, Avg loss: 0.671057

Fold: 3
Current loss: 0.520403, [    0/45000]
Current loss: 0.723666, [ 4500/45000]
Current loss: 0.576373, [ 9000/4

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [12]:
print("VGG16")
perform_kfold_run(getVGGModel, VGG_Weights_Path)

Downloading: "https://download.pytorch.org/models/vgg13-19584684.pth" to /root/.cache/torch/hub/checkpoints/vgg13-19584684.pth
100%|██████████| 508M/508M [00:13<00:00, 38.6MB/s]


Fold: 1
Current loss: 2.430878, [    0/45000]
Current loss: 0.914583, [ 4500/45000]
Current loss: 0.761437, [ 9000/45000]
Current loss: 0.846669, [13500/45000]
Current loss: 0.819225, [18000/45000]
Current loss: 0.650042, [22500/45000]
Current loss: 0.598005, [27000/45000]
Current loss: 0.782234, [31500/45000]
Current loss: 0.541521, [36000/45000]
Current loss: 0.770884, [40500/45000]
For Current Run: Accuracy: 74.2%, Avg loss: 0.720432

Fold: 2
Current loss: 0.688775, [    0/45000]
Current loss: 0.710284, [ 4500/45000]
Current loss: 0.450511, [ 9000/45000]
Current loss: 0.848126, [13500/45000]
Current loss: 0.741117, [18000/45000]
Current loss: 0.561102, [22500/45000]
Current loss: 0.612241, [27000/45000]
Current loss: 0.829419, [31500/45000]
Current loss: 0.632014, [36000/45000]
Current loss: 0.384975, [40500/45000]
For Current Run: Accuracy: 74.8%, Avg loss: 0.693802

Fold: 3
Current loss: 0.601989, [    0/45000]
Current loss: 0.752194, [ 4500/45000]
Current loss: 0.700041, [ 9000/4

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (16): 

In [13]:
print("ResNet 50")
perform_kfold_run(getResNetModel, RESNET50_Weights_Path)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 67.4MB/s]


Fold: 1
Current loss: 2.322500, [    0/45000]
Current loss: 1.892672, [ 4500/45000]
Current loss: 1.657476, [ 9000/45000]
Current loss: 1.464953, [13500/45000]
Current loss: 1.362168, [18000/45000]
Current loss: 1.181401, [22500/45000]
Current loss: 1.124771, [27000/45000]
Current loss: 1.325378, [31500/45000]
Current loss: 0.972042, [36000/45000]
Current loss: 1.027969, [40500/45000]
For Current Run: Accuracy: 69.2%, Avg loss: 1.035364

Fold: 2
Current loss: 1.247169, [    0/45000]
Current loss: 1.175277, [ 4500/45000]
Current loss: 0.781128, [ 9000/45000]
Current loss: 1.078870, [13500/45000]
Current loss: 1.083685, [18000/45000]
Current loss: 0.738509, [22500/45000]
Current loss: 0.913797, [27000/45000]
Current loss: 0.963798, [31500/45000]
Current loss: 0.875646, [36000/45000]
Current loss: 0.836832, [40500/45000]
For Current Run: Accuracy: 68.8%, Avg loss: 0.930910

Fold: 3
Current loss: 1.114416, [    0/45000]
Current loss: 0.987599, [ 4500/45000]
Current loss: 0.857164, [ 9000/4

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

## Section 5: Test the model

Write a function to test the model using testloader.

In [14]:
AlexNet = getAlexNetModel()
AlexNet.load_state_dict(torch.load(AlexNet_Weights_Path))
AlexNet.eval()
AlexNet.to(DEVICE)
accuracy = compute_accuracy(AlexNet, testloader)
print(accuracy)

For Current Run: Accuracy: 81.8%, Avg loss: 0.514072

81.8


In [15]:
VGG = getVGGModel()
VGG.load_state_dict(torch.load(VGG_Weights_Path))
VGG.eval()
VGG.to(DEVICE)
accuracy = compute_accuracy(VGG, testloader)
print(accuracy)

For Current Run: Accuracy: 81.8%, Avg loss: 0.522855

81.81


In [16]:
ResNet50 = getResNetModel()
ResNet50.load_state_dict(torch.load(RESNET50_Weights_Path))
ResNet50.eval()
ResNet50.to(DEVICE)
accuracy = compute_accuracy(ResNet50, testloader)
print(accuracy)

For Current Run: Accuracy: 76.4%, Avg loss: 0.699790

76.42999999999999
